<div style="text-align: center;">
    <h1 style="font-style: italic;">[Auto]Stitching Photo Mosaics</h1>
</div>
<div style="display: flex; justify-content: space-around; align-items: center;">
    <div style="text-align: center;">
        <img src="./media/intro.jpg" alt="Image 1" style="width: 600px;">
    </div>
</div>

## ***IMAGE WARPING and MOSAICING***

&emsp;&emsp;The goal of this assignment is to explore different aspects of image warping through a practical and exciting application—image mosaicing. In this project, I worked with photographs and created image mosaics by performing several key operations: image registration, projective warping and blending. Along the way, I learned how to compute homographies and used them to warp and align images.

### ***Recover Homographies***

&emsp;&emsp;Before warping the images into alignment, we need to recover the parameters of the transformation between each pair of images. In our case, the transformation is a homography: $\mathbf{p^{\prime}} = \mathbf{Hp}$, where $\mathbf{H}$ is a $3 \times 3$ matrix with $8$ degrees of freedom (lower right corner is a scaling factor and can be set to $1$), that is
$$
\begin{bmatrix}
x^{\prime} \\
y^{\prime} \\
1 \\
\end{bmatrix} = \begin{bmatrix}
a & b & c \\
d & e & f \\
g & h & 1 \\
\end{bmatrix} \cdot \begin{bmatrix}
x \\
y \\
1 \\
\end{bmatrix}
$$
&emsp;&emsp;We could convert the equation above into th form of $\mathbf{Ah} = \mathbf{b}$, where $\mathbf{h} = \begin{bmatrix}
a, b, c, d, e, f, g, h \\
\end{bmatrix}^{\top}$. Then, for each pair of corresponding points, we have
$$
\begin{bmatrix}
x_{i} & y_{i} & 1 & 0 & 0 & 0 & -x_{i}x_{i}^{\prime} & -y_{i}x_{i}^{\prime} \\
0 & 0 & 0 & x_{i} & y_{i} & 1 & -x_{i}y_{i}^{\prime} & -y_{i}y_{i}^{\prime} \\
\end{bmatrix} \cdot \mathbf{h} = \begin{bmatrix}
x_{i}^{\prime} \\
y_{i}^{\prime} \\
\end{bmatrix}
$$
&emsp;&emsp;To solve this, at least 4 pairs of corresponding points should be chosen. However, with only four points, the homography recovery will be very unstable and prone to noise. Therefore more than 4 correspondences should be provided producing an overdetermined system which should be solved using least-squares. Then, we could solve this using least-squares,
$$
\mathbf{h} = \arg\min_{\mathbf{h}} \|\mathbf{Ah} - \mathbf{b}\|^{2} = (\mathbf{A}^{\top}\mathbf{A})^{-1}\mathbf{A}^{\top}\mathbf{b}
$$

### ***Warp the Images***

&emsp;&emsp;To ensure that the warped image will not go beyond the canvas, I first add borders with 0, i.e. black borders, to them and modify the position of the corresponding points as well, which should be related to the size of border added. With the homography matrix $\mathbf{H}$, I could warp the source image to match the view direction of the target image.

&emsp;&emsp;I choose to perform inverse warping. I compute the corresponding coordinates of pixels in the source image of each pixels in the destination image, which could be calculated according to the homography matrix $\mathbf{H}$. Then, I use `scipy.interpolate.griddata` to perform interpolation.

### ***Image Rectification***

&emsp;&emsp;Here are two examples of image rectification.

<div style="display: flex; justify-content: space-around; align-items: center;">
    <div style="text-align: center;">
        <p style="font-weight: bold;">Original Phone</p>
        <img src="./media/phone_lean.jpg" alt="Image 1" style="height: 260px;">
    </div>
    <div style="text-align: center;">
        <p style="font-weight: bold;">Rectifies Phone</p>
        <img src="./media/phone.jpg" alt="Image 2" style="height: 260px;">
    </div>
    <div style="text-align: center;">
        <p style="font-weight: bold;">Original Shoe Box</p>
        <img src="./media/puma_lean.jpg" alt="Image 1" style="height: 260px;">
    </div>
    <div style="text-align: center;">
        <p style="font-weight: bold;">Rectified Shoe Box</p>
        <img src="./media/puma.jpg" alt="Image 2" style="height: 260px;">
    </div>
</div>

### ***Blend the images into a mosaic***

&emsp;&emsp;To blend the images into a mosaic, I choose the central image as a target for the other images to warp to. Once they are warpped, build alpha mask for them following the logic below:
$$
    \alpha = \text{logical}(\text{dtrans}1 > \text{dtrans}2)
$$
$$
    \alpha = \text{blurred}
$$
&emsp;&emsp;That is, set the part where transformed distance of image 1 is larger as $1$, and then blur the mask.

&emsp;&emsp;Then I could blend all image together with Laplacian stacks to make a mosaic. I also tried Laplacian pyramids as well, but the results of stack are better. I use alpha mask to blend images and the target image. To blend the bended images together, I just use the trivial mask that divide the image into left and right part since the target is set to center.

&emsp;&emsp;The results are shown below.

#### ***Li Ka Shing Center***

&emsp;&emsp;These are images taken near the Li Ka Shing Center.
<div style="display: flex; justify-content: space-around; align-items: center;">
    <div style="text-align: center;">
        <img src="./media/pts_building.jpg" alt="Image 1" style="width: 800px;">
    </div>
</div>

&emsp;&emsp;Warp them into the direction of central image:
<div style="display: flex; justify-content: space-around; align-items: center;">
    <div style="text-align: center;">
        <img src="./media/warped_building.jpg" alt="Image 1" style="width: 800px;">
    </div>
</div>

&emsp;&emsp;Blend them together,
<div style="display: flex; justify-content: space-around; align-items: center;">
    <div style="text-align: center;">
        <img src="./media/mosaic_building.jpg" alt="Image 1" style="width: 400px;">
    </div>
</div>

#### ***Street***

&emsp;&emsp;These are images of a student housing at Hearst Street.
<div style="display: flex; justify-content: space-around; align-items: center;">
    <div style="text-align: center;">
        <img src="./media/pts_street.jpg" alt="Image 1" style="width: 800px;">
    </div>
</div>

&emsp;&emsp;Warp them into the direction of central image:
<div style="display: flex; justify-content: space-around; align-items: center;">
    <div style="text-align: center;">
        <img src="./media/warped_street.jpg" alt="Image 1" style="width: 800px;">
    </div>
</div>

&emsp;&emsp;Blend them together,
<div style="display: flex; justify-content: space-around; align-items: center;">
    <div style="text-align: center;">
        <img src="./media/mosaic_street.jpg" alt="Image 1" style="width: 400px;">
    </div>
</div>

### ***Valley Life Sciences Building***

&emsp;&emsp;I took these cool images at Valley Life Sciences Building.
<div style="display: flex; justify-content: space-around; align-items: center;">
    <div style="text-align: center;">
        <img src="./media/pts_model.jpg" alt="Image 1" style="width: 800px;">
    </div>
</div>

&emsp;&emsp;Warp them into the direction of central image:
<div style="display: flex; justify-content: space-around; align-items: center;">
    <div style="text-align: center;">
        <img src="./media/warped_model.jpg" alt="Image 1" style="width: 800px;">
    </div>
</div>

&emsp;&emsp;Blend them together,
<div style="display: flex; justify-content: space-around; align-items: center;">
    <div style="text-align: center;">
        <img src="./media/mosaic_model.jpg" alt="Image 1" style="width: 400px;">
    </div>
</div>